## Projet Machine learning : emission de CO2 des véhicules commercial en France 


In [37]:
import pandas as pd
df = pd.read_csv("data/CO2csv.csv", sep=";", encoding="latin1")
print("\nLe jeu de donnée à été charger avec succes !\n")

# Aperçu des données
#print('\n',df.head())
print(' ')

# Informations sur les colonnes
print(df.info())

# Statistiques descriptives
print('\n',df.describe())

# Dimensions du dataset
print('\n',f"Dimensions : {df.shape}")
print('')

print('\n',df.describe(include='object'))


Le jeu de donnée à été charger avec succes !

 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55044 entries, 0 to 55043
Data columns (total 30 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   lib_mrq            55044 non-null  object 
 1   lib_mod_doss       55044 non-null  object 
 2   lib_mod            55044 non-null  object 
 3   dscom              55044 non-null  object 
 4   cnit               55044 non-null  object 
 5   tvv                55044 non-null  object 
 6   cod_cbr            55044 non-null  object 
 7   hybride            55044 non-null  object 
 8   puiss_admin_98     55044 non-null  int64  
 9   puiss_max          55044 non-null  object 
 10  typ_boite_nb_rapp  55044 non-null  object 
 11  conso_urb          55001 non-null  object 
 12  conso_exurb        55001 non-null  object 
 13  conso_mixte        55010 non-null  object 
 14  co2                55010 non-null  float64
 15  co_typ_1           54

## analyse des données manquante et partie netoyage

In [38]:
# analyse des données manquant puis netoyage echellonée de c'est donnée
print(" ")
print(len(df))

# suppression de column vide dans le jeux de données 
df.drop(['Unnamed: 26', 'Unnamed: 27', 'Unnamed: 28', 'Unnamed: 29', 'date_maj', 'hc'], axis=1, inplace=True)

# enlévé les variable qui ne serviront a rien dans la prediction ou qui risque d'engendre du bruit.

df.drop(['cnit', 'tvv', 'puiss_admin_98', 'co_typ_1', 'dscom', 'hcnox', 'nox'], axis=1, inplace=True)

# suppression des colomn de consomation car il donne déjà des information sur les emission de C02 
df.drop(['conso_urb', 'conso_exurb', 'conso_mixte', 'ptcl'], axis=1, inplace=True)
# cars CO2 = conso_mixte * constante, donc le modeles risque d'apprendre le mauvais calcule même si le resultat sera bon 

# car le nom du vehicule n'affecte pas les emission de CO2 donc c'est donner risque de nous ménee a un état de sur-apprentissage 
df.drop(['lib_mod', 'lib_mod_doss'], axis=1, inplace=True)

print(df.isna().sum())



 
55044
lib_mrq               0
cod_cbr               0
hybride               0
puiss_max             0
typ_boite_nb_rapp     0
co2                  34
masse_ordma_min       0
masse_ordma_max       0
champ_v9             61
Carrosserie           0
gamme                 0
dtype: int64


 - les trois derniere column ne servent a rien, elle peuvent donc être enlevé
 - la colonne HC contient un trop grand nombre de valeur manquante pour pouvoir être emputé (on va l'enlevé)
 - la colonne date_maj idem que la colonne HC
 - la colonne HCNOX depend depend de la colonne HC qui elle est manquante (on va faire un ensemble avec HCNOX et un sans)
 - le reste des valeur seront êmputé 

### suppression des colonne avec trop de val manquante

float64
object
object


### netoyage des differente variable retenu

In [39]:
## netoyage de CO2

# 1. Nettoyage CO2
df['co2'] = pd.to_numeric(df['co2'], errors='coerce')
df['co2'] = df['co2'].fillna(df['co2'].mean())

# 2. Extraction de caractéristiques (Feature Engineering)

df['euro_num'] = df['champ_v9'].str.extract(r'(\d+)$').astype(float)

# 3. Remplissage des nouveaux NaN

df['euro_num'] = df['euro_num'].fillna(df['euro_num'].median())

df.drop('champ_v9', axis=1, inplace=True)


In [40]:
# Création d'une masse moyenne et suppression des anciennes
df['masse_moyenne'] = (df['masse_ordma_min'] + df['masse_ordma_max']) / 2
df.drop(['masse_ordma_min', 'masse_ordma_max'], axis=1, inplace=True)

## separation des type de boite et du nombre de vitesse


In [41]:
# 1. On sépare la colonne en utilisant l'espace comme délimiteur

split_boite = df['typ_boite_nb_rapp'].str.split(' ', expand=True)

# 2. On assigne les résultats aux nouvelles colonnes
df['type_boite'] = split_boite[0]

# 3. On convertit le nombre de rapports en numérique
# errors='coerce' transformera les éventuels textes bizarres en NaN
df['nb_rapports'] = pd.to_numeric(split_boite[1], errors='coerce')

# 4. Suppression de l'ancienne colonne pour éviter les doublons
df.drop('typ_boite_nb_rapp', axis=1, inplace=True)

# Vérification
print(df[['type_boite', 'nb_rapports']].head())


  type_boite  nb_rapports
0          M          6.0
1          M          6.0
2          M          6.0
3          M          6.0
4          M          6.0


In [ ]:
# emputation de nb_rapports
df['nb_rapports'] = df['nb_rapports'].fillna(df['nb_rapports'].mode()[0])
df['nb_rapports'] = df['nb_rapports'].astype('Int64')

## Hot encoding des variable qualitatif


lib_mrq          0
cod_cbr          0
hybride          0
puiss_max        0
co2              0
Carrosserie      0
gamme            0
euro_num         0
masse_moyenne    0
type_boite       0
nb_rapports      0
dtype: int64


In [52]:
print(df.dtypes)
print(df.isna().sum())

lib_mrq           object
cod_cbr           object
hybride           object
puiss_max         object
co2              float64
Carrosserie       object
gamme             object
euro_num         float64
masse_moyenne    float64
type_boite        object
nb_rapports        int32
dtype: object
lib_mrq          0
cod_cbr          0
hybride          0
puiss_max        0
co2              0
Carrosserie      0
gamme            0
euro_num         0
masse_moyenne    0
type_boite       0
nb_rapports      0
dtype: int64


In [ ]:
# errors='coerce' transforme ce qui n'est pas un chiffre en NaN
df['puiss_max'] = pd.to_numeric(df['puiss_max'], errors='coerce')



In [14]:
df.fillna(df['puiss_max'].mode()[0], axis=1, inplace=True)

In [17]:
#suppression des doublons 
df = df.drop_duplicates()
